In [1]:
import urllib.request as urllib2
import json
import pandas as pd
import numpy as np
import time as t
import datetime
import itertools
import os
from scipy.sparse import lil_matrix
from pprint import pprint

First code blocks are focused on just gathering data for a mesh of atlanta. Essentially 4 closest points are gathered
4|V| = |E|. Then, the adj matrix (A) of edges to be collected are collected for array of coords are collected and dumped into csv. $A_{ij}$ implies info going from i to j. What edges to use is dependent only on A.

In [2]:
"""
INPUTS:
    url: a request url
OUTPUTS: 
    the data returned by calling that url
"""
def request_data_from_url(url, num_attempts = 3):
    req = urllib2.Request(url)
    success = False
    for epoch in range(num_attempts):
        if success:
            return response.read()
        try: 
            #open the url
            response = urllib2.urlopen(req)
            
            #200 is the success code for http
            if response.getcode() == 200:
                success = True
        except Exception as e:
            #if we didn't get a success, then print the error and wait 5 seconds before trying again
            print(e)
            t.sleep(5)

            print("Error for URL %s: %s" % (url, datetime.datetime.now()))
            print("Retrying...")

In [3]:
"""
INPUTS:
    api_key: authentication to GMaps that we're allowed to request this data
    origin: lat,long of origin
    destination: lat,long of destination
    transport_mode: mode of travel, options are driving, walking, bicycling, and transit
    units_type: unit type of the output, options are metric (kilometers and meters) and imperial (miles and feet)
OUTPUTS
    (time_traffic,time,distance,origin_addr,destination_addr)
    real time
    usual time
    distance
"""
def scrape_gmaps_data(api_key, origin, destination, transport_mode, units_type = 'metric'):
    
    #we want to scrape the google maps website
    site = 'https://maps.googleapis.com/maps/api/'
    
    #we want to use the distance matrix service
    service = 'distancematrix/json?'
    
    #input origin and destination from the user 
    locations = f"origins={origin[0]},{origin[1]}&destinations={destination[0]},{destination[1]}&departure_time=now"

    #input api key from user
    key = '&key=%s' % (api_key)
    
    #specify transportation mode
    mode = '&mode=%s' % (transport_mode)
    
    #specify units
    units = '&units=%s' % (units_type)
    
    #construct request url
    request_url = site + service + locations + mode + units + key
    
    # get data from api
    data = json.loads(request_data_from_url(request_url))
    
    # origin address
    oa = data['origin_addresses'][0]
    
    # destination address
    da = data['destination_addresses'][0]
    
    # check if the request failed
    if data['rows'][0]['elements'][0]['status'] == 'ZERO_RESULTS':
        return (-1,-1,-1, oa, da)
    
    #extract travel avg time from response
    time = data['rows'][0]['elements'][0]['duration']['value']
    
    #extract distance from response
    dist = data['rows'][0]['elements'][0]['distance']['value'] 
    
    #extract travel real time from response
    if transport_mode == 'driving':
        realtime = data['rows'][0]['elements'][0]['duration_in_traffic']['value']
    else:
        realtime = time
    
    return (realtime, time, dist, oa, da)

Proof that this works! And gives us what we want

In [25]:
with open("supersecretapikey.txt","r") as fh:
    api_key = fh.readlines()[0]
origin = (33.776360,-84.397824) # Campus Center
destination = (33.772428,-84.392709) #Bobby Dodd

# note only driving has a duration_in_traffic so both times are the same
print(scrape_gmaps_data(api_key, origin, destination, 'driving'))
print(scrape_gmaps_data(api_key, origin, destination, 'walking'))
print(scrape_gmaps_data(api_key, origin, destination, 'transit')) # note this does walking to cover holes
print(scrape_gmaps_data(api_key, origin, destination, 'bicycling'))

(226, 248, 1093, '788 Research Dr NW, Atlanta, GA 30313, USA', '177 N Ave NW, Atlanta, GA 30313, USA')
(728, 728, 998, '788 Research Dr NW, Atlanta, GA 30313, USA', '177 N Ave NW, Atlanta, GA 30313, USA')
(728, 728, 998, '788 Research Dr NW, Atlanta, GA 30313, USA', '177 N Ave NW, Atlanta, GA 30313, USA')
(191, 191, 1017, '788 Research Dr NW, Atlanta, GA 30313, USA', '177 N Ave NW, Atlanta, GA 30313, USA')


In [22]:
"""
INPUTS:
    adjacency matrix: sparse boolean matrix representing which edges to query for
    location_matrix: np.array floating points of size |N|x2 (Longitude, Latitude), 
        could also be of shape |NM|x2 for a grid input
    period: how often to query in minutes
    number_of_queries: how many times to query
    api_key: api key for your gcp account 
OUTPUTS
    Pandas df
"""
def graph_query(location_matrix, adjacency_matrix, period, number_of_queries, api_key, transport_mode):
    # basic checks 
    if location_matrix.shape[0] != adjacency_matrix.shape[0]:
        raise ValueError("length of locations is not the same as size of adjacency matrix")
    if adjacency_matrix.shape[0] != adjacency_matrix.shape[1]:
        raise ValueError("invalid adjacency matrix shape")
    
    # see how many elements there are in the matrix
    mag_vset = 0
    cx = adjacency_matrix.tocoo()
    for i,j,v in zip(cx.row, cx.col, cx.data): mag_vset += 1
    mag_vset *= number_of_queries
    
    # make the pandas dataframe
    df = pd.DataFrame(np.zeros(shape=(mag_vset,11)))
    df.columns = ["Time","Actual Query Time","Origin Latitude","Origin Longitude","Origin Address",
                  "Destination Latitude","Destination Longitude","Destination Address",
                  "Distance","Traffic Time","Normal Time"]
    
    # go through all elements with the query
    df_index = 0
    for qi in range(number_of_queries):
        time = datetime.datetime.now()
        for i,j,v in zip(cx.row, cx.col, cx.data):
            # get origin and destination
            origin = (location_matrix[i,0], location_matrix[i,1])
            destination = (location_matrix[i,0],location_matrix[j,1])
            # Actually Query GCP (Distance Matrix)
            (realtime_travel, time_travel, dist, oa, da) = scrape_gmaps_data(api_key, origin, destination, transport_mode)
            # Add all data to dataframe
            df.iloc[df_index,0], df.iloc[df_index,1] = time, datetime.datetime.now()
            df.iloc[df_index,2],df.iloc[df_index,3],df.iloc[df_index,4] = origin[0], origin[1], oa
            df.iloc[df_index,5],df.iloc[df_index,6],df.iloc[df_index,7] = destination[0], destination[1], da
            df.iloc[df_index,8], df.iloc[df_index,9], df.iloc[df_index,10] = realtime_travel, time_travel, dist
            df_index += 1
        # wait to get to whole period before new queries
        if qi != number_of_queries - 1:
            t.sleep(period * 60 - (datetime.datetime.now() - time).total_seconds())
    return df

In [26]:
location_matrix = np.array([[33.776360,-84.397824],
                           [33.772428,-84.392709]])
adjacency_matrix = lil_matrix((2,2),dtype = '?')
adjacency_matrix[0,1] = True
adjacency_matrix[1,0] = True

df = graph_query(location_matrix,adjacency_matrix,3/60,3,api_key,'walking')
df

,Time,Actual Query Time,Origin Latitude,Origin Longitude,Origin Address,Destination Latitude,Destination Longitude,Destination Address,Distance,Traffic Time,Normal Time
0,2019-11-17 17:38:33.393739,2019-11-17 17:38:33.554846,33.776360,-84.397824,"788 Research Dr NW, Atlanta, GA 30313, USA",33.776360,-84.392709,"166 5th St NE, Atlanta, GA 30308, USA",474.0,474.0,664.0
1,2019-11-17 17:38:33.393739,2019-11-17 17:38:33.763937,33.772428,-84.392709,"177 N Ave NW, Atlanta, GA 30313, USA",33.772428,-84.397824,"645 State St NW, Atlanta, GA 30313, USA",551.0,551.0,708.0
2,2019-11-17 17:38:36.394638,2019-11-17 17:38:36.620100,33.776360,-84.397824,"788 Research Dr NW, Atlanta, GA 30313, USA",33.776360,-84.392709,"166 5th St NE, Atlanta, GA 30308, USA",474.0,474.0,664.0
3,2019-11-17 17:38:36.394638,2019-11-17 17:38:36.812807,33.772428,-84.392709,"177 N Ave NW, Atlanta, GA 30313, USA",33.772428,-84.397824,"645 State St NW, Atlanta, GA 30313, USA",551.0,551.0,708.0
4,2019-11-17 17:38:39.394832,2019-11-17 17:38:39.624220,33.776360,-84.397824,"788 Research Dr NW, Atlanta, GA 30313, USA",33.776360,-84.392709,"166 5th St NE, Atlanta, GA 30308, USA",474.0,474.0,664.0
5,2019-11-17 17:38:39.394832,2019-11-17 17:38:39.822739,33.772428,-84.392709,"177 N Ave NW, Atlanta, GA 30313, USA",33.772428,-84.397824,"645 State St NW, Atlanta, GA 30313, USA",551.0,551.0,708.0


In [106]:
# center of the mesh
center_mesh = [33.776360,-84.397824]
# how many rows to make mesh
num_rows = 20
# how many columns
num_columns = 20

# the mesh as a numpy array
mesh = np.zeros(shape = (num_rows, num_columns, 2))

# distance between them
delta = .005
lat = []
lon = []

# make all vertices from a mesh
for i,j in itertools.product(range(num_rows), range(num_columns)):
    mesh[i,j,0] = center_mesh[0] + (i-num_rows/2)*delta
    mesh[i,j,1] = center_mesh[1] + (j-num_columns/2)*delta

# get linearized indices
index = np.zeros(shape = (num_rows, num_columns), dtype = np.int32)
for i,j in itertools.product(range(num_rows), range(num_columns)):
    index[i,j] = num_columns * i + j

# magnitude of vector set
mag_vset = num_rows * num_columns
    
# linearize the mesh into a location_matrix
location_matrix = np.zeros(shape = (mag_vset,2))
for i,j in itertools.product(range(num_rows), range(num_columns)):
    location_matrix[index[i,j],:] = mesh[i,j,:]

# add only the "mesh" coordinates (one hop neighbors)
adjacency_matrix = lil_matrix((mag_vset,mag_vset),dtype = '?')

index

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19],
       [ 20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
         33,  34,  35,  36,  37,  38,  39],
       [ 40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59],
       [ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
         73,  74,  75,  76,  77,  78,  79],
       [ 80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
         93,  94,  95,  96,  97,  98,  99],
       [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
        113, 114, 115, 116, 117, 118, 119],
       [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
        133, 134, 135, 136, 137, 138, 139],
       [140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
        153, 154, 155, 156, 157, 158, 159],
       [160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 1

In [78]:
location_matrix[:5]
location_matrix.shape

(400, 2)

In [119]:
#### Take home project: Add all the "mesh" edges to the adjacency list in O(N) time. This assumes the location matrix ###
#### is formatted linearly from 2D mesh as defined above####

def mesh_population(adjacency_matrix):
    for i,j in itertools.product(range(num_rows), range(num_columns)):
        n1 = index[i,j]
        if i != j:
            if i > 0:
                n2 = index[i-1,j]
                adjacency_matrix[n1,n2] = True
                adjacency_matrix[n2,n1] = True
            if i < num_rows - 1:
                n2 = index[i+1,j]
                adjacency_matrix[n1,n2] = True
                adjacency_matrix[n2,n1] = True
            if j > 0:
                n2 = index[i,j-1]
                adjacency_matrix[n1,n2] = True
                adjacency_matrix[n2,n1] = True
            if j < num_columns - 1:
                n2 = index[i,j+1]
                adjacency_matrix[n1,n2] = True
                adjacency_matrix[n2,n1] = True
    return adjacency_matrix

adjacency_matrix = mesh_population(adjacency_matrix)
for i in range(adjacency_matrix.shape[0]):
    if adjacency_matrix[i,i]:
        print(i)
adjacency_matrix[:5,:5].todense()

matrix([[0., 1., 0., 0., 0.],
        [1., 0., 1., 0., 0.],
        [0., 1., 0., 1., 0.],
        [0., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0.]], dtype=float32)

In [108]:
# num of edges asked for
print(np.sum(adjacency_matrix.todense()))

# roughly what is should be 2 for both directions 2 for neighbors n for rows m for cols
print(num_rows * num_columns * 2 * 2)

1520
1600


In [120]:
#### Take home project: Implement csv_graph method, which goes from a csv to a graph representation of the query.
#### Assume that the dataset was created as above. Please make the adj. mat. sparse (look above for example). The values
#### on the edges should not be booleans but instead time magnitude. The default will be traffic time BUT you should support
#### using normal time and distance (use that optional param). This should be efficient!!!####
import operator

def csv_graph(csv, value = "Traffic Time"):
    
    df = pd.read_csv(csv)
    
    locations = set()
    for row in range(df.shape[0]):
        loc = tuple([df["Origin Latitude"][row],df["Origin Longitude"][row]])
        locations.add(loc)
    locations = list(locations)
    
    num_vertex = len(locations)
    location_array = np.zeros(shape = (num_vertex, 2))
    adjacency_matrix = lil_matrix((num_vertex,num_vertex), dtype = np.float32)

    locations.sort(key = operator.itemgetter(0, 1))
    for i in range(len(locations)):
        location_array[i,0] = locations[i][0]
        location_array[i,1] = locations[i][1]
        
    for row in range(df.shape[0]):
        loc_1 = tuple([df["Origin Latitude"][row],df["Origin Longitude"][row]])
        loc_2 = tuple([df["Destination Latitude"][row],df["Destination Longitude"][row]])
        i = np.where((location_array == loc_1).all(axis=1))[0][0]
        j = np.where((location_array == loc_2).all(axis=1))[0][0]
        val = df[value][row]
        adjacency_matrix[i,j] = df[value][row]
        
    return location_array, adjacency_matrix

In [121]:
location_array, adjacency_matrix = csv_graph("../dataset/dataset.csv")
location_array[:5]

array([[ 33.72636 , -84.447824],
       [ 33.72636 , -84.442824],
       [ 33.72636 , -84.437824],
       [ 33.72636 , -84.432824],
       [ 33.72636 , -84.427824]])

In [122]:
adjacency_matrix[:100,:100].todense()

matrix([[  0., 401.,   0., ...,   0.,   0.,   0.],
        [441.,   0., 767., ...,   0.,   0.,   0.],
        [  0., 706.,   0., ...,   0.,   0.,   0.],
        ...,
        [  0.,   0.,   0., ...,   0., 399.,   0.],
        [  0.,   0.,   0., ..., 331.,   0., 351.],
        [  0.,   0.,   0., ...,   0., 320.,   0.]], dtype=float32)

In [118]:
df = graph_query(location_matrix,adjacency_matrix,3,1,api_key,'walking')

<urlopen error [Errno 11001] getaddrinfo failed>


KeyboardInterrupt: 

In [33]:
path = os.path.join(os.getcwd(), "..")
path = os.path.join(path, "dataset")
path = os.path.join(path, "dataset.csv")
#df.to_csv(path)